In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cbook, rc_params_from_file, rcParamsDefault

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn import metrics

#classifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error
from sklearn import svm

from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *

In [ ]:
train = pd.read_csv(r"/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv")
test = pd.read_csv(r"/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_test.csv")

In [ ]:
train.describe().transpose()

In [ ]:
train.keys()

In [ ]:
train.info()

company_size와 company_type에서 null이 많으므로 두 변수는 제외하려고 함

In [ ]:
train=pd.DataFrame(train, columns=['enrollee_id','city', 'city_development_index', 'gender',
                                     'relevent_experience', 'enrolled_university', 'education_level',
                                     'major_discipline', 'experience', 'last_new_job', 'training_hours', 'target'])

In [ ]:
test=pd.DataFrame(test, columns=['enrollee_id','city', 'city_development_index', 'gender',
                                 'relevent_experience', 'enrolled_university', 'education_level',
                                 'major_discipline', 'experience', 'last_new_job', 'training_hours'])

In [ ]:
test.info()

null이 있는 행 삭제

In [ ]:
train.isnull().sum()/len(train) # count Null

In [ ]:
# drop missing data
train.dropna(inplace=True)
test.dropna(inplace=True)
# train_d1=train[['enrolled_university','education_level','experience','last_new_job']].dropna()
# train_d1.isnull().sum()
train.info()

In [ ]:
test.info()

In [ ]:
test.isnull().sum()

In [ ]:
train.info()

In [ ]:
train

In [ ]:
train = train.drop_duplicates('enrollee_id', keep='last')

object로 들어온 데이터를 정수형으로 바꿔줌 

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for x in train:
    if train[x].dtypes=='object':
        train[x] = le.fit_transform(train[x])
# we have transformed all the non numeric data columns into data columns
# this method applies 0,1.. classification to different value types.

In [ ]:
train.corr()['target'].sort_values()

In [ ]:
train.describe().transpose()

In [ ]:
train.info()

target의 분포 확인

In [ ]:
plt.hist(train['target'], label='Target');
plt.legend();

enrollee_id, city 두 변수는 임의로 제외

In [ ]:
train_x=pd.DataFrame(train, columns=['city_development_index', 'gender',
                                     'relevent_experience', 'enrolled_university', 'education_level',
                                     'major_discipline', 'experience','last_new_job', 'training_hours'])

In [ ]:
train_x.shape

heatmap으로 변수간의 상관관계 확인

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
corr = train_x.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(240,10,as_cmap=True),
            square=True, ax=ax)

In [ ]:
train_y=train.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X_train)

In [ ]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [ ]:
model = models.Sequential()

model.add(layers.Dense(9, activation='relu'))
model.add(layers.Dense(3, activation='relu'))
model.add(layers.Dense(3, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy')
#              , # Target 0 또는 1로 예측
#              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,validation_data=(X_test,y_test),epochs=10, batch_size=64,verbose=0)

In [ ]:
print(model.summary())

In [ ]:
model_loss = pd.DataFrame(model.history.history)
model_loss.plot()

생성된 모델을 test data에 넣어서 예상값 확인하기

In [ ]:
testt=pd.DataFrame(test, columns=['city_development_index', 'gender',
                                  'relevent_experience', 'enrolled_university', 'education_level',
                                  'major_discipline', 'experience','last_new_job', 'training_hours'])

In [ ]:
testt.info()

In [ ]:
# object로 들어온 데이터를 정수형으로 바꿔줌 
le = LabelEncoder()
for x in testt:
    if testt[x].dtypes=='object':
        testt[x] = le.fit_transform(testt[x])

In [ ]:
testt.info()

In [ ]:
test['pred'] = model.predict_classes(testt)
test.pred.describe()

In [ ]:
test.info()

pred가 1로 나타난 행 확인

In [ ]:
test[test.pred==1]

In [ ]:
predictions = model.predict_classes(X_test)

In [ ]:
predictions

아래 값들로 모델이 변수간의 관계를 적절히 나타내지 못하는 것을 확인하여 target의 category가 더 많거나 연속형 변수인 경우가 나았을 것이라 생각됨

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score, r2_score

In [ ]:
mean_squared_error(y_test,predictions)

In [ ]:
np.sqrt(mean_squared_error(y_test,predictions))

In [ ]:
explained_variance_score(y_test,predictions)

In [ ]:
r2_score(y_test,predictions)